In [1]:
import numpy as np
import pandas as pd
from scipy import stats

%precision 3
np.random.seed(1111)

In [2]:
df = pd.read_csv('/Users/yuya/Desktop/Python_Static/python_stat_sample/data/ch11_potato.csv')
sample = np.array(df['重さ'])
sample

array([122.02, 131.73, 130.6 , 131.82, 132.05, 126.12, 124.43, 132.89,
       122.79, 129.95, 126.14, 134.45, 127.64, 125.68])

In [4]:
# 標本平均を求める
s_mean = np.mean(sample)
s_mean

128.4507142857143

In [7]:
# 母平均は130gという仮定で仮説検定を行う
# ※フライドポテトの母集団は正規分布で、その分散は9とする

# 標本平均XがP(X<=x) = 0.05 を満たすxを考える
rv = stats.norm(130, np.sqrt(9/14))
rv.isf(0.95)

128.68118313069039

In [8]:
# 上記の計算により、標本平均が128.68g以下になる確率は
# 5%である事が分かった
# Aさんの抽出した標本平均が128.451gなので、
# これは5%しか起きない珍しい出来事だと考えられる

# 標準化した値で再計算してみる
# 検定統計量
z = (s_mean - 130) / np.sqrt(9/14)
z

-1.932298779026813

In [10]:
# 臨界値を求める
rv = stats.norm()
rv.isf(0.95)

-1.6448536269514722

In [11]:
# 検定統計量 < 臨界値
# だったので、帰無仮説は棄却され、「平均は130gより少ない」という結論になる

# p値を使った仮説検定
# p値を求める
rv.cdf(z)

0.026661319523126635